In [1]:
import os
os.chdir('../..')

In [2]:
from platform import python_version
print(python_version())

3.7.9


In [3]:
import numpy as np
import pandas as pd

import tensorflow as tf

import matplotlib.pyplot as plt

from modules.datgan import DATWGAN

import networkx as nx
import json
import beepy

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [4]:
dataset = 'Chicago'

In [5]:
df = pd.read_csv('../data/{}/data.csv'.format(dataset), index_col=False)

In [6]:
df.head()

,choice,travel_dow,trip_purpose,distance,hh_vehicles,hh_size,hh_bikes,hh_descr,hh_income,gender,age,license,education_level,work_status,departure_time
0,drive,7,HOME_OTHER,23.42579,2,2,0,2,6,1,66,1.0,6,FTE,9.333333
1,drive,7,OTHER,1.71259,2,2,0,2,6,1,66,1.0,6,FTE,12.083333
2,drive,7,HOME_OTHER,21.77887,2,2,0,2,6,1,66,1.0,6,FTE,15.500000
3,drive,7,SHOPPING,2.02603,2,2,0,2,6,1,66,1.0,6,FTE,17.500000
4,drive,7,SHOPPING,0.87691,2,2,0,2,6,1,66,1.0,6,FTE,18.250000


In [7]:
if dataset is 'Chicago':
    continuous_columns = ["distance", "age", "departure_time"]
elif dataset is 'LPMC':
    continuous_columns = ['start_time_linear', 'age', 'distance', 'dur_walking', 'dur_cycling', 'dur_pt_access', 'dur_pt_rail', 'dur_pt_bus', 'dur_pt_int', 'dur_driving', 'cost_transit', 'cost_driving_fuel', 'driving_traffic_percent']

In [8]:
# personalised graph
graph = nx.DiGraph()

if dataset is 'Chicago':
    graph.add_edges_from([
        ("age", "license"),
        ("age", "education_level"),
        ("gender", "work_status"),
        ("education_level", "work_status"),
        ("education_level", "hh_income"),
        ("work_status", "hh_income"),
        ("hh_income", "hh_descr"),
        ("hh_income", "hh_size"),
        ("hh_size", "hh_vehicles"),
        ("hh_size", "hh_bikes"),
        ("work_status", "trip_purpose"),
        ("trip_purpose", "departure_time"),
        ("trip_purpose", "distance"),
        ("travel_dow", "choice"),
        ("distance", "choice"),
        ("departure_time", "choice"),
        ("hh_vehicles", "choice"),
        ("hh_bikes", "choice"),
        ("license", "choice"),
        # Non necessary links
        ("education_level", "hh_size"),
        ("work_status", "hh_descr"),
        ("work_status", "hh_size"),
        ("hh_income", "hh_bikes"),
        ("hh_income", "hh_vehicles"),
        ("trip_purpose", "choice")
    ])
elif dataset is 'LPMC':
    graph.add_edges_from([
        ("travel_year", "survey_year"),
        ("travel_date", "day_of_week"),
        ("day_of_week", "purpose"),
        ("purpose", "start_time_linear"),
        ("purpose", "cost_driving_con_charge"),
        ("purpose", "distance"),
        ("day_of_week", "driving_traffic_percent"),
        ("day_of_week", "cost_driving_con_charge"),
        ("start_time_linear", "driving_traffic_percent"),
        ("start_time_linear", "cost_driving_con_charge"),
        ("driving_traffic_percent", "cost_driving_con_charge"),
        ("female", "driving_license"),
        ("age", "bus_scale"),
        ("age", "car_ownership"),
        ("age", "driving_license"),
        ("age", "faretype"),
        ("driving_license", "car_ownership"),
        ("car_ownership", "fueltype"),
        ("fueltype", "cost_driving_con_charge"),
        ("fueltype", "cost_driving_fuel"),
        ("distance", "cost_driving_fuel"),
        ("distance", "dur_driving"),
        ("distance", "dur_walking"),
        ("distance", "dur_cycling"),
        ("distance", "dur_pt_access"),
        ("distance", "dur_pt_rail"),
        ("distance", "dur_pt_bus"),
        ("distance", "dur_pt_int"),
        ("dur_pt_bus", "cost_transit"),
        ("dur_pt_rail", "cost_transit"),
        ("pt_n_interchanges", "dur_pt_int"),
        ("pt_n_interchanges", "cost_transit"),
        ("faretype", "cost_transit"),
        ("bus_scale", "cost_transit"),
        ("car_ownership", "travel_mode"),
        ("age", "travel_mode"),
        ("cost_driving_con_charge", "travel_mode"),
        ("driving_traffic_percent", "travel_mode"),
        ("female", "travel_mode"),
        ("purpose", "travel_mode"),
        ("cost_transit", "travel_mode"),
        ("cost_driving_fuel", "travel_mode"),
        ("dur_driving", "travel_mode"),
        ("dur_walking", "travel_mode"),
        ("dur_cycling", "travel_mode"),
        ("dur_pt_access", "travel_mode"),
        ("dur_pt_rail", "travel_mode"),
        ("dur_pt_bus", "travel_mode"),
        ("dur_pt_int", "travel_mode")
    ])
    graph.add_node("travel_month")

In [9]:
output_folder = '../output/' + dataset + '/DATWGAN_30/'

In [10]:
datgan = DATWGAN(continuous_columns, max_epoch=1000, batch_size=500, output=output_folder, gpu=0)

In [11]:
test = datgan.fit(df, graph)

[1022 14:17:23 @DATGAN.py:155] Found preprocessed data
[1022 14:17:23 @DATGAN.py:163] Preprocessed data have been loaded!



[1022 14:17:23 @input_source.py:222] Setting up the queue 'QueueInput/input_queue' for CPU prefetching ...








Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
[1022 14:17:23 @DATGANModel.py:209] Creating cell for age (in-edges: 0)
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
[1022 14:17:23 @registry.py:126] gen/LSTM/00/FC input: [500, 100]

Instructions for updating:
Please use `layer.__call__` method instead.
[1022 14:17:23 @registry.py:134] gen/LSTM/00/FC output: [500, 100]
[1022 14:17:23 @registry.py:126] gen/LSTM/00/FC2_val input: [500, 100]
[1022 14:17:23 @registry.py:134] gen/LSTM/00/FC2_val output: [500, 6]
[1022 14:17:23 @regist

[1022 14:17:24 @registry.py:126] gen/LSTM/06/FC3 input: [500, 8]
[1022 14:17:24 @registry.py:134] gen/LSTM/06/FC3 output: [500, 100]
[1022 14:17:24 @DATGANModel.py:209] Creating cell for hh_income (in-edges: 2)
[1022 14:17:24 @registry.py:126] gen/LSTM/07/FC_inputs input: [500, 200]
[1022 14:17:24 @registry.py:134] gen/LSTM/07/FC_inputs output: [500, 100]
[1022 14:17:24 @registry.py:126] gen/LSTM/07/FC_attentions input: [500, 200]
[1022 14:17:24 @registry.py:134] gen/LSTM/07/FC_attentions output: [500, 100]
[1022 14:17:24 @registry.py:126] gen/LSTM/07/FC_lstm_state_0 input: [500, 200]
[1022 14:17:24 @registry.py:134] gen/LSTM/07/FC_lstm_state_0 output: [500, 100]
[1022 14:17:24 @registry.py:126] gen/LSTM/07/FC_lstm_state_1 input: [500, 200]
[1022 14:17:24 @registry.py:134] gen/LSTM/07/FC_lstm_state_1 output: [500, 100]
[1022 14:17:24 @registry.py:126] gen/LSTM/08/FC input: [500, 100]
[1022 14:17:24 @registry.py:134] gen/LSTM/08/FC output: [500, 100]
[1022 14:17:24 @registry.py:126] gen

[1022 14:17:24 @registry.py:134] gen/LSTM/19/FC3 output: [500, 100]
[1022 14:17:24 @DATGANModel.py:209] Creating cell for choice (in-edges: 7)
[1022 14:17:24 @registry.py:126] gen/LSTM/20/FC_inputs input: [500, 700]
[1022 14:17:24 @registry.py:134] gen/LSTM/20/FC_inputs output: [500, 100]
[1022 14:17:24 @registry.py:126] gen/LSTM/20/FC_attentions input: [500, 700]
[1022 14:17:24 @registry.py:134] gen/LSTM/20/FC_attentions output: [500, 100]
[1022 14:17:24 @registry.py:126] gen/LSTM/20/FC_lstm_state_0 input: [500, 700]
[1022 14:17:24 @registry.py:134] gen/LSTM/20/FC_lstm_state_0 output: [500, 100]
[1022 14:17:24 @registry.py:126] gen/LSTM/20/FC_lstm_state_1 input: [500, 700]
[1022 14:17:24 @registry.py:134] gen/LSTM/20/FC_lstm_state_1 output: [500, 100]
[1022 14:17:24 @registry.py:126] gen/LSTM/21/FC input: [500, 100]
[1022 14:17:24 @registry.py:134] gen/LSTM/21/FC output: [500, 100]
[1022 14:17:24 @registry.py:126] gen/LSTM/21/FC2 input: [500, 100]
[1022 14:17:24 @registry.py:134] gen/

[1022 14:17:28 @base.py:209] Setup callbacks graph ...






[1022 14:17:28 @summary.py:46] [MovingAverageSummary] 5 operations in collection 'MOVING_SUMMARY_OPS' will be run with session hooks.
[1022 14:17:28 @summary.py:93] Summarizing collection 'summaries' of size 6.




[1022 14:17:29 @base.py:230] Creating the session ...


Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.



[1022 14:17:31 @base.py:236] Initializing the session ...
[1022 14:17:31 @sessinit.py:114] Restoring checkpoint from ../output/Chicago/DATWGAN_30/model\model-104999 ...
INFO:tensorflow:Restoring parameters from ../output/Chicago/DATWGAN_30/model\model-104999
[1022 14:17:31 @base.py:243] Graph Finalized.


[1022 14:17:31 @concurrency.py:38] Starting EnqueueThread QueueInput/input_queue ...
[1022 14:17:31 @steps.py:124] Start training with global_step=104999
[1022 14:17:32 @monitor.py:347] Found existing JSON inside ../output/Chicago/DATWGAN_30/logs, will append to 

 95%|##############################################################################7    |166/175[00:08<00:00,26.77it/s]

...

[1022 15:11:37 @base.py:285] Epoch 999 (global_step 209789) finished, time:6.31 seconds.
[1022 15:11:37 @saver.py:79] Model saved to ../output/Chicago/DATWGAN_30/model\model-209789.
[1022 15:11:37 @monitor.py:467] GAN_loss/d_loss: -0.02383


[1022 15:11:37 @monitor.py:467] GAN_loss/g_loss: 0.23661
[1022 15:11:37 @monitor.py:467] GAN_loss/gradient_penalty: 0.0048075
[1022 15:11:37 @monitor.py:467] GAN_loss/gradient_rms: 0.99816
[1022 15:11:37 @monitor.py:467] GAN_loss/kl_div: 0.054863
[1022 15:11:37 @monitor.py:467] QueueInput/queue_size: 50
[1022 15:11:37 @base.py:275] Start Epoch 1000 ...


100%|###################################################################################|175/175[00:06<00:00,27.73it/s]

[1022 15:11:43 @base.py:285] Epoch 1000 (global_step 209999) finished, time:6.31 seconds.
[1022 15:11:43 @saver.py:79] Model saved to ../output/Chicago/DATWGAN_30/model\model-209999.


[1022 15:11:44 @monitor.py:467] GAN_loss/d_loss: -0.032628
[1022 15:11:44 @monitor.py:467] GAN_loss/g_loss: 0.24609
[1022 15:11:44 @monitor.py:467] GAN_loss/gradient_penalty: 0.004826
[1022 15:11:44 @monitor.py:467] GAN_loss/gradient_rms: 1.0004
[1022 15:11:44 @monitor.py:467] GAN_loss/kl_div: 0.061365
[1022 15:11:44 @monitor.py:467] QueueInput/queue_size: 50
[1022 15:11:44 @base.py:289] Training has finished!

[1022 15:11:44 @DATGANModel.py:209] Creating cell for age (in-edges: 0)
[1022 15:11:44 @DATGANModel.py:209] Creating cell for gender (in-edges: 0)
[1022 15:11:44 @DATGANModel.py:209] Creating cell for travel_dow (in-edges: 0)
[1022 15:11:44 @DATGANModel.py:209] Creating cell for license (in-edges: 1)
[1022 15:11:44 @DATGANModel.py:209] Creating cell for education_level (in-edges: 1)
[1022 15:11:44 @DATGANModel.py:209] Creating cell for work_status (in-edges: 2)
[1022 15:11:44 @DATGANModel.py:209] Creating cell for hh_income (in-edges: 2)
[1022 15:11:44 @DATGANModel.py:209] Creat

In [12]:
datgan.save('trained', force=True)

[1022 15:12:08 @DATGAN.py:347] Model saved successfully.


In [13]:
beepy.beep(6)